In [1]:
!pip install -q kaggle
import json

data = {"username":"tunkedsaro","key":"ff3d00dc680ff94caf6f8d4d9bcd3168"}
json_data = json.dumps(data)

# Write the JSON string to a file
file_path = "/content/kaggle.json"
with open(file_path, 'w') as json_file:
    json_file.write(json_data)

!mkdir ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/ #copy api key ---- depend on your directory -- my directory is .../colab/..
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets list
!kaggle competitions download -c brain-motor-imagery-classification

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 441, in run
    conflicts = self._determine_conflicts(to_install)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 

In [2]:
pip install braindecode

<hr>

In [3]:
from scipy.signal import butter, filtfilt, iirnotch, freqz
import os
import numpy as np

In [4]:
# Band-pass filter
def bandpass_filter(data, lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    y = filtfilt(b, a, data)
    return y

# Notch filter
def notch_filter(data, cut_freq, fs, Q=30):
    nyq = 0.5 * fs
    freq = cut_freq / nyq
    b, a = iirnotch(freq, Q)
    y = filtfilt(b, a, data)
    return y

def filter03050(Ipsignal):
    sampling_rate = 250  # Hz
    # fft_signal = np.fft.fft(Ipsignal)
    # frequencies = np.fft.fftfreq(len(Ipsignal), 1/sampling_rate)
    filtered_signal = bandpass_filter(Ipsignal, 1, 30, sampling_rate)          # band-pass filter 0-30 Hz
    filtered_signal = notch_filter(filtered_signal, 50, sampling_rate)       # notch filter 50 Hz
    return filtered_signal

def unique_name(Datapath):
    all_name = os.listdir(Datapath)
    new = []
    for un in all_name:
        i = un.replace("_data_time_series.npy","").replace("_data_time_stamps.npy","").replace("_label_time_series.npy","").replace("_label_time_stamps.npy","")
        new.append(i)
        unique_name = list(set(new))
    return unique_name

def check_different_duration(label_time_stamps):
    keep_time = []
    start_time = 0
    for current_time in list(label_time_stamps):
        keep_time.append(current_time-start_time)
        start_time = current_time
    keep_time = keep_time[1:]
    avgtime = sum(keep_time)/len(keep_time)
    stat = True
    for etime in keep_time:
        if 6.8 <= etime <= 7.5:
            stat = True
        else:
            stat = False
            break
    return stat,avgtime

file_path = "/content/train/train"
unique_name = unique_name(file_path)

In [5]:
unique_name.remove("s1_d2_p002_003")          # sameple ห่าง
unique_name.remove("s1_d2_p006_008")          # label != 30
unique_name.remove("s2_d2_p002_006")          # label != 30

In [6]:
def filter_one(windows,percent_pass):
    percentage_ones = (np.sum(windows==1)/len(windows))
    if percentage_ones >= percent_pass:
        return True
    return False

In [7]:
data_dataset  = []
label_dataset = []
for name in unique_name:
    # print(name)
    signals_series    = np.load(os.path.join(file_path,name+"_data_time_series.npy"))
    times             = np.load(os.path.join(file_path,name+"_data_time_stamps.npy"))
    label_time_series = np.load(os.path.join(file_path,name+"_label_time_series.npy"))
    label_time_stamps = np.load(os.path.join(file_path,name+"_label_time_stamps.npy"))
    signals = signals_series[:,:5]
    windows = signals_series[:,16]
    if not(filter_one(windows,1)):            # กรอง คุณภาพ 0,1
        continue
    stat,avgtime = check_different_duration(label_time_stamps)
    if not(stat):                               # กรอง ระยะห่างของ time_stamp
        continue
    if filter!=None:                            # Apply filter
        filtered_signal = []
        Dim = signals.shape[1]
        # print(Dim)
        for i in range(Dim):
            # print(f"{i} : {signals[:,i]}")
            filtered_signal.append(filter03050(signals[:,i]))
        signals = np.array(filtered_signal).T
    # sample_point = int(round(avgtime))
    # datapoint = 250 * sample_point                   # 250 Hz * 7 s
    for index in range(30):
        # start_point = index * datapoint
        # end_point = (index + 1) * datapoint
        # if end_point <= signals.shape[0]:  # Check to ensure slice is within the data range
        #     data_dataset.append(signals[start_point:end_point])
        # else:
        #     break
        sn = signals[1750*i:1750*(i+1)]
        if sn.shape[0]!=30 and sn.shape[1]!=5:
            print(name,"error")
        data_dataset.append(sn)
    label_dataset.append(label_time_series)
data_dataset = np.stack(data_dataset)

label_dataset = np.concatenate(label_dataset)
label_dataset = label_dataset.flatten().tolist()

In [8]:
# Demension check
data_dataset.shape

(1530, 1750, 5)

In [9]:
label_dataset = np.array(label_dataset)
label_dataset

array([120, 110, 150, ..., 150, 120, 110])

In [10]:
label_dataset.shape

(1530,)

<hr>

In [11]:
from braindecode.models.util import models_dict
print(f'All the Braindecode models:\n{list(models_dict.keys())}')

All the Braindecode models:
['ATCNet', 'Deep4Net', 'DeepSleepNet', 'EEGConformer', 'EEGITNet', 'EEGInception', 'EEGInceptionERP', 'EEGInceptionMI', 'EEGNetv1', 'EEGNetv4', 'EEGResNet', 'HybridNet', 'ShallowFBCSPNet', 'SleepStagerBlanco2020', 'SleepStagerChambon2018', 'SleepStagerEldele2021', 'TCN', 'TIDNet', 'USleep']


In [12]:
from braindecode.models import ShallowFBCSPNet              # Import the model from braindecode * There are many model that available
import mne
import numpy as np

In [13]:
# Train spect config
model = ShallowFBCSPNet(
    n_chans   = 5,
    n_times   = 1750,
    n_outputs = 3,
    final_conv_length='auto',
)
print(model)

Layer (type (var_name):depth-idx)        Input Shape               Output Shape              Param #                   Kernel Shape
ShallowFBCSPNet (ShallowFBCSPNet)        [1, 5, 1750]              [1, 3]                    --                        --
├─Ensure4d (ensuredims): 1-1             [1, 5, 1750]              [1, 5, 1750, 1]           --                        --
├─Rearrange (dimshuffle): 1-2            [1, 5, 1750, 1]           [1, 1, 1750, 5]           --                        --
├─CombinedConv (conv_time_spat): 1-3     [1, 1, 1750, 5]           [1, 40, 1726, 1]          9,040                     --
├─BatchNorm2d (bnorm): 1-4               [1, 40, 1726, 1]          [1, 40, 1726, 1]          80                        --
├─Expression (conv_nonlin_exp): 1-5      [1, 40, 1726, 1]          [1, 40, 1726, 1]          --                        --
├─AvgPool2d (pool): 1-6                  [1, 40, 1726, 1]          [1, 40, 111, 1]           --                        [75, 1]
├─Express

/usr/local/lib/python3.10/dist-packages/braindecode/models/base.py:180: UserWarning: LogSoftmax final layer will be removed! Please adjust your loss function accordingly (e.g. CrossEntropyLoss)!
  warnings.warn("LogSoftmax final layer will be removed! " +


In [14]:
data_dataset.shape

(1530, 1750, 5)

In [15]:
data_dataset = data_dataset.reshape(data_dataset.shape[0], 5, 1750)
data_dataset.shape

(1530, 5, 1750)

In [16]:
label_dataset.shape

(1530,)

In [17]:
label_dataset = np.array(label_dataset)
label_dataset

array([120, 110, 150, ..., 150, 120, 110])

In [18]:
# Map value
mapping = {110: 0,
           120: 1,
           150: 2}

mapped_label = np.vectorize(mapping.get)(label_dataset)
label = np.array(mapped_label)
label

array([1, 0, 2, ..., 2, 1, 0])

In [19]:
info = mne.create_info(
    ch_names = ['c1','c2','c3','c4','c5'], sfreq=250., ch_types="eeg"
)
# data_dataset  # 100 epochs, 8 channels, 7 seconds (@250Hz)
epochs = mne.EpochsArray(data_dataset, info=info)


Not setting metadata
1530 matching events found
No baseline correction applied
0 projection items activated


In [20]:
from skorch.dataset import ValidSplit
from braindecode import EEGClassifier

In [21]:
net = EEGClassifier(
    'ShallowFBCSPNet',
    module__final_conv_length='auto',
    max_epochs=10,
    train_split=ValidSplit(0.2),
    # To train a neural network you need validation split, here, we use 20%.
)
# Train model
net.fit(epochs, label)

/usr/local/lib/python3.10/dist-packages/braindecode/models/base.py:180: UserWarning: LogSoftmax final layer will be removed! Please adjust your loss function accordingly (e.g. CrossEntropyLoss)!
  warnings.warn("LogSoftmax final layer will be removed! " +


  epoch    train_loss    valid_acc    valid_loss      dur
-------  ------------  -----------  ------------  -------
      1        8.6574       0.3170        3.6769  27.8001
      2        4.9064       0.2876        5.1919  18.3609
      3        3.7825       0.2712        2.3111  16.8733
      4        2.7244       0.2745        1.7899  16.8645
      5        2.2870       0.3235        2.4488  16.9007
      6        2.3771       0.2680        1.4336  18.9839
      7        2.0592       0.3431        1.6927  18.0222
      8        2.1309       0.3072        2.2488  17.2103
      9        2.1316       0.3464        1.6881  17.0615
     10        1.9749       0.3007        1.9241  16.9641


<class 'braindecode.classifier.EEGClassifier'>[initialized](
  module_=============================================================================================================================================
  Layer (type (var_name):depth-idx)        Input Shape               Output Shape              Param #                   Kernel Shape
  ShallowFBCSPNet (ShallowFBCSPNet)        [1, 5, 1750]              [1, 3]                    --                        --
  ├─Ensure4d (ensuredims): 1-1             [1, 5, 1750]              [1, 5, 1750, 1]           --                        --
  ├─Rearrange (dimshuffle): 1-2            [1, 5, 1750, 1]           [1, 1, 1750, 5]           --                        --
  ├─CombinedConv (conv_time_spat): 1-3     [1, 1, 1750, 5]           [1, 40, 1726, 1]          9,040                     --
  ├─BatchNorm2d (bnorm): 1-4               [1, 40, 1726, 1]          [1, 40, 1726, 1]          80                        --
  ├─Expression (conv_nonlin_exp): 

<hr>

<h1>Test model</h1>

In [22]:
from scipy.signal import butter, filtfilt, iirnotch, freqz
import os

In [23]:
data_dataset  = []
label_dataset = []
file_path = "/content/train/train"
unique_name = ["s1_d2_p002_002"]
for name in unique_name:
    print(name)
    signals           = np.load(os.path.join(file_path,name+"_data_time_series.npy"))
    times             = np.load(os.path.join(file_path,name+"_data_time_stamps.npy"))
    label_time_series = np.load(os.path.join(file_path,name+"_label_time_series.npy"))
    label_time_stamps = np.load(os.path.join(file_path,name+"_label_time_stamps.npy"))
    signals = signals[:,:5]
    print(signals.shape)
    stat,avgtime = check_different_duration(label_time_stamps)
    if not(stat):
        continue
    # print("Avg time",self.avgtime)
    if filter!=None:
        filtered_signal = []
        Dim = signals.shape[1]
        print(Dim)
        for i in range(Dim):
            # print(f"{i} : {signals[:,i]}")
            filtered_signal.append(filter03050(signals[:,i]))
        signals = np.array(filtered_signal).T
    sample_point = int(round(avgtime))
    datapoint = 250 * sample_point                   # 250 Hz * 7 s
    for index in range(30):
        start_point = index * datapoint
        end_point = (index + 1) * datapoint
        if end_point <= signals.shape[0]:  # Check to ensure slice is within the data range
            data_dataset.append(signals[start_point:end_point])
        else:
            break
    label_dataset.append(label_time_series)
data_dataset = np.stack(data_dataset)
print(data_dataset.shape)
label_dataset = np.concatenate(label_dataset)
label_dataset = label_dataset.flatten().tolist()

s1_d2_p002_002
(53714, 5)
5
(30, 1750, 5)


In [24]:
data_dataset = data_dataset.reshape(30,5,1750)
data_dataset.shape

(30, 5, 1750)

In [25]:
test_epochs = mne.EpochsArray(data_dataset,info=info)

Not setting metadata
30 matching events found
No baseline correction applied
0 projection items activated


In [26]:
# Get predictions from the trained model
predictions = net.predict(test_epochs)
probabilities = net.predict_proba(test_epochs)

In [27]:
# Map value
mapping = {110: 0, 120: 1, 150: 2}

mapped_label = np.vectorize(mapping.get)(label_dataset)
print(mapped_label)

[1 0 2 2 1 0 1 0 2 2 1 0 1 2 0 2 1 0 1 0 2 2 1 0 2 1 0 2 1 0]


In [28]:
print("Predictions:", predictions)
print("Probabilities:", probabilities)

Predictions: [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
Probabilities: [[2.55428720e-03 1.64398500e-07 9.97445583e-01]
 [7.32635185e-02 2.76540413e-06 9.26733673e-01]
 [8.86575785e-03 1.10296205e-05 9.91123199e-01]
 [5.05082496e-02 3.57462994e-07 9.49491382e-01]
 [1.02549642e-02 6.01029171e-07 9.89744484e-01]
 [5.88529091e-03 2.90102946e-08 9.94114697e-01]
 [1.92669481e-02 4.96658670e-09 9.80733037e-01]
 [5.45883738e-02 6.14332905e-08 9.45411623e-01]
 [5.67389652e-02 2.31679891e-07 9.43260789e-01]
 [1.68121353e-01 7.03323451e-07 8.31877947e-01]
 [1.01833669e-02 1.95202812e-10 9.89816666e-01]
 [3.49477120e-03 1.95459959e-09 9.96505260e-01]
 [4.59094252e-03 4.68333416e-09 9.95409071e-01]
 [1.01962071e-02 1.00209583e-08 9.89803791e-01]
 [5.42405900e-03 5.13525533e-09 9.94575977e-01]
 [6.06307155e-03 9.72222391e-09 9.93936956e-01]
 [3.73598421e-03 1.08254277e-07 9.96263921e-01]
 [3.83584239e-02 5.37799860e-06 9.61636186e-01]
 [1.17900163e-01 1.25425754e-06 8.82098615e-01

In [29]:
Cm = 0
for i,j in zip(mapped_label,predictions):
    print(i,j,i==j)
    if i==j:
        Cm+=1
print(f"{Cm}/{len(predictions)}")

1 2 False
0 2 False
2 2 True
2 2 True
1 2 False
0 2 False
1 2 False
0 2 False
2 2 True
2 2 True
1 2 False
0 2 False
1 2 False
2 2 True
0 2 False
2 2 True
1 2 False
0 2 False
1 2 False
0 2 False
2 2 True
2 2 True
1 2 False
0 2 False
2 2 True
1 2 False
0 2 False
2 2 True
1 2 False
0 2 False
10/30


<hr>

<h1>Prediction</h1>

In [30]:
import pandas as pd

In [31]:
sample_submission = pd.read_csv("/content/sample_submission.csv")
sample_submission

,id,predict
0,a0dfb1f6-a76b-404f-a04a-b61e955ce1ef,110.0
1,51afd110-7556-4a7d-b6d2-68e46256f267,150.0
2,eb229854-1a14-4e1f-8e52-470edbec47c9,150.0
3,8fa74efe-f269-415e-a968-1a92e7c66fe6,NaN
4,7b209881-a1dd-4aa3-a06a-037908b07bfa,NaN
...,...,...
475,69e84db3-0019-4e38-90fc-968f45747a16,NaN
476,d18d7d80-78fa-402a-ba1c-e35d72382b18,NaN
477,0a532391-dd42-413f-ab15-798960de95d6,NaN
478,0c02ad4f-de15-48cb-ad31-0925e69bfe97,NaN


In [32]:
test_file_path = '/content/test'
for ss in sample_submission["id"][3:]:
    file_name = os.path.join(test_file_path,ss+".npy")
    signals   = np.load(file_name)[:,:5]
    filtered_signal = []
    Dim = signals.shape[1]
    for i in range(Dim):
        filtered_signal.append(filter03050(signals[:,i]))
    signals = np.array(filtered_signal).T
    signals_reshape = signals.T.reshape(1, 5, signals.shape[0])

    test_epochs = mne.EpochsArray(signals_reshape,info=info)
    predictions = net.predict(test_epochs)
    sample_submission.loc[sample_submission['id'] == ss, 'predict'] = predictions[0]

Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1 matching events found
No baselin

In [33]:
sample_submission['predict'] = sample_submission['predict'].apply(lambda ip: "110" if ip == 0 else "120" if ip == 1 else "150" if ip == 2 else str(int(ip)))

In [34]:
sample_submission

,id,predict
0,a0dfb1f6-a76b-404f-a04a-b61e955ce1ef,110
1,51afd110-7556-4a7d-b6d2-68e46256f267,150
2,eb229854-1a14-4e1f-8e52-470edbec47c9,150
3,8fa74efe-f269-415e-a968-1a92e7c66fe6,110
4,7b209881-a1dd-4aa3-a06a-037908b07bfa,110
...,...,...
475,69e84db3-0019-4e38-90fc-968f45747a16,110
476,d18d7d80-78fa-402a-ba1c-e35d72382b18,110
477,0a532391-dd42-413f-ab15-798960de95d6,110
478,0c02ad4f-de15-48cb-ad31-0925e69bfe97,110


In [35]:
i110 = 0
i120 = 0
i150 = 0
for item in list(sample_submission['predict']):
    if item == '110':
        i110+=1
    elif item == '120':
        i120+=1
    elif item == '150':
        i150+=1

In [36]:
# Labels and values for the bar plot
labels = ['110', '120', '150']
values = [i110, i120, i150]

# Create the bar plot
plt.bar(labels, values, color=['blue', 'green', 'red'])

# Add title and labels to the plot
plt.xlabel('Variables')
plt.ylabel('Values')
plt.grid(alpha=0.3)

# Display the plot
plt.show()

NameError: name 'plt' is not defined

In [ ]:
# sample_submission.to_csv("/content/brainwave 0.1.4.csv", index=False)